<a href="https://colab.research.google.com/github/KyriakiGeorgiou/KyriakiGeorgiou/blob/main/CV_Checker_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U https://gradio-builds.s3.amazonaws.com/a4559af9e1bc6226fc0130d9ab32cc161e99efdd/gradio-3.36.1-py3-none-any.whl

  Using cached https://gradio-builds.s3.amazonaws.com/a4559af9e1bc6226fc0130d9ab32cc161e99efdd/gradio-3.36.1-py3-none-any.whl (10.1 MB)


In [ ]:
%%capture
!pip install transformers==4.30.2
!pip install spacy_transformers==1.2.5
!pip install spacy

In [ ]:
!pip show spacy_transformers

Name: spacy-transformers
Version: 1.2.5
Summary: spaCy pipelines for pre-trained BERT and other transformers
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, spacy, spacy-alignments, srsly, torch, transformers
Required-by: 


In [ ]:
!pip show transformers

Name: transformers
Version: 4.30.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: spacy-transformers


In [ ]:
!pip show spacy

Name: spacy
Version: 3.5.0
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi
Required-by: en-core-web-lg, en-core-web-sm, fastai, PassivePy, spacy-transformers


In [ ]:
%%capture
!pip install -r https://raw.githubusercontent.com/mitramir55/PassivePy/main/PassivePyCode/PassivePySrc/requirements_lg.txt
!pip install PassivePy

In [ ]:
%%capture
!pip install pymupdf

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
import spacy
import spacy_transformers

In [ ]:
from PassivePySrc import PassivePy
import os
import fitz
import re
from collections import Counter

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import gradio as gr
from gradio.inputs import Textbox, File

In [ ]:
!python -m spacy validate

In [ ]:
!python -m spacy download en_core_web_lg

2023-08-02 08:01:06.309528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 3.4.0
    Uninstalling en-core-web-lg-3.4.0:
      Successfully uninstalled en-core-web-lg-3.4.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:

def read_resume(resume_file):

  doc = fitz.open(resume_file)

  text = " "
  page_count = 0
  for page in doc:
    text = text + str(page.get_text())
    page_count += 1

  word_count = len(text.split())

  return text,word_count,page_count

def clean_CV_simple(resume):

    cleaned_text = resume.strip()
    cleaned_text = cleaned_text.replace("•", "")  # Remove bullet points
    cleaned_text = cleaned_text.replace("·","")
    cleaned_text = cleaned_text.replace("_", "")
    cleaned_text = cleaned_text.replace("➢", "")
    cleaned_text = cleaned_text.replace("| ","")
    cleaned_text = cleaned_text.replace(",","").replace("(","").replace(".","").replace(")","").replace("‘","").replace(":","").replace("&","").replace(";","").replace("/","").replace("-"," ")
    cleaned_text = cleaned_text.replace("etc.","")

    cleaned_text = re.sub(r'\band\b', '', cleaned_text, flags=re.IGNORECASE)# Remove standalone "and" while preserving it if it is part of a word

    cleaned_text = " ".join(cleaned_text.split())  # Remove unnecessary spaces

    return cleaned_text

# 1.Quantifying Impact
def get_percentages(cleaned_resume):
  percentages = re.findall(r'(\d+)%', cleaned_resume)
  percentages_list = []
  for percentage in percentages:
      percentages_list.append(percentage)
  return percentages_list

# Helper function for 2
def get_verbs(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.pos_ == 'VERB']

lemmatizer = WordNetLemmatizer()

# 2. Action Verb Use - Deduct marks
def action_verb_use(resume_verbs):

  verbs_to_avoid = ['assisted', 'led', 'oversaw', 'utilized' ,'worked','helped','supported']
  lemmatized_verbs_to_avoid = []

  for word in verbs_to_avoid:
      pos = wordnet.VERB
      lemma = lemmatizer.lemmatize(word, pos=pos)
      lemmatized_verbs_to_avoid.append(lemma)

  resume_verbs_to_avoid = [word for word in resume_verbs if word in lemmatized_verbs_to_avoid]
  return resume_verbs_to_avoid


strong_verbs = ['Accelerated', 'Achieved', 'Attained' ,'Completed', 'Conceived',
                'Convinced', 'Discovered' ,'Doubled','Effected','Eliminated' ,'Expanded',
                'Expedited' ,'Founded' ,'Improved','Increased','Initiated','Innovated',
                'Introduced','Invented','Launched','Mastered','Overcame','Overhauled','Pioneered'
                ,'Reduced','Resolved','Revitalised','Spearheaded','Strengthened','Transformed',
                'Upgraded','Tripled','Addressed','Advised','Arranged' ,'Authored','Co-authored'
                ,'Co-ordinated','Communicated', 'Corresponded','Counselled','Developed',
                'Demonstrated','Directed','Drafted','Enlisted','Facilitated','Formulated'
                ,'Guided','Influenced','Interpreted','Interviewed','Instructed','Lectured'
                ,'Liased','Mediated','Moderated','Motivated','Negotiated','Persuaded'
                ,'Presented','Promoted','Proposed','Publicised','Recommended','Reconciled'
                ,'Recruited','Resolved','Taught','Trained','Translated','Arranged','Budgeted',
                'Composed','Conceived','Conducted','Controlled','Co-ordinated','Eliminated','Improved'
                ,'Investigated','Itemised','Modernised','Operated','Organised','Planned','Prepared'
                ,'Processed','Produced','Redesigned','Reduced','Refined','Researched','Resolved','Reviewed'
                ,'Revised','Scheduled','Simplified','Solved','Streamlined','Transformed','Examined'
                ,'Revamped','Acted','Conceptualised','Created','Customised','Designed','Developed'
                ,'Directed','Redesigned','Established','Fashioned','Illustrated','Instituted'
                ,'Integrated','Performed','Planned','Proved','Revised','Revitalised','Set up',
                'Shaped','Streamlined','Structured','Tabulated','Validated', 'Approved',
                'Arranged','Catalogued','Classified','Collected','Compiled','Dispatched','Executed','Filed'
                ,'Generated','Implemented','Inspected','Monitored','Operated','Ordered','Organised','Prepared'
                ,'Processed','Purchased','Recorded','Retrieved','Screened','Specified','Systematised'
                ,'administered', 'allocated', 'analysed', 'appraised', 'audited', 'balanced', 'budgeted',
                'calculated', 'computed', 'developed', 'managed', 'planned', 'projected', 'researched',
                'restructured', 'modelled']

# 2.1 Strong Verb Use
def strong_verbs_resume(resume_verbs):
  lemmatized_strong_verbs = []

  for word in strong_verbs:
      pos = wordnet.VERB
      lemma = lemmatizer.lemmatize(word.lower(), pos=pos)
      lemmatized_strong_verbs.append(lemma)


  resume_strong_verbs = [word for word in resume_verbs if word in lemmatized_strong_verbs]
  return resume_strong_verbs

# 3. Accompishments - Avoid Responsibility oriented langauge - Deduct marks
def accomplishments(clean_resume):
  responsibility_oriented = ['Demonstrated success in', 'Proven results', 'Successful', 'success',
                            'Results-driven', 'results-oriented', 'Excellent communication skills',
                            'Responsible for', 'Duties included', 'Seasoned', 'Accomplished', 'Worked with',
                            'served as']


  resume_responsibility_oriented = [phrase for phrase in responsibility_oriented if phrase.lower() in clean_resume.lower()]
  return resume_responsibility_oriented

# 4. Repetition
def repetition(resume_verbs):

  duplicates = [item for item, count in Counter(resume_verbs).items() if count > 1]
  return duplicates

# 5. Resume Length

# def resume_length(resume_file):

#     doc = fitz.open(re)
#     text = " "
#     page_count = 0
#     for page in doc:
#         text = text + str(page.get_text())
#         page_count +=1

#     word_count = len(text.split())


#     return word_count,page_count

# 6. Filler Words

def filler_words(clean_resume):

  filler_words = ['various', 'in order to', 'several', 'as well as', 'so that', 'eventually','resourcefully'
                  'carried out', 'multiple','quickly','successfully', 'as needed','efficiently',
                  'always', 'frequently', 'actively', 'thoroughly', 'efficiently', 'diligently',
                  'proficiently', 'significantly']

  filler_words_resume = [phrase for phrase in filler_words if phrase in clean_resume]

  return filler_words_resume

# 7. Total Bullet Points
def total_bullet_points(raw_resume):
  bullet_symbols = ['•', '·', '➢']
  bullet_points_count = sum(raw_resume.count(symbol) for symbol in bullet_symbols)
  return bullet_points_count

# Load NER model
nlp = spacy.load('/content/drive/MyDrive/xml_model')

# 8. Bullet Points Length

def count_bullet_word_lengths(text):
    bullet_word_counts = []
    bullet_symbols = ['•', '-', '*']

    for symbol in bullet_symbols:
        segments = text.split(symbol)
        word_counts = [len(re.findall(r'\b\w+\b', segment)) for segment in segments[1:]]
        bullet_word_counts.append(word_counts)

    return bullet_word_counts

# Load Passive Voice Model
passivepy = PassivePy.PassivePyAnalyzer(spacy_model = "en_core_web_lg")

# 9. Active Voice
def detect_passive_voice(clean_resume):
  passive_voice = passivepy.match_text(clean_resume, full_passive=True, truncated_passive=True)['passive_count']
  return passive_voice.item()

# 10. Personal Pronouns
def find_personal_pronouns(clean_resume):
    personal_pronouns = ['I', 'me', 'you', 'he', 'she', 'it', 'we', 'they', 'him', 'her', 'us', 'them','mine','ours']
    resume_personal_pronouns = re.findall(r'\b(?:%s)\b' % '|'.join(personal_pronouns), clean_resume, flags=re.IGNORECASE)
    return resume_personal_pronouns

buzzwords = ['Ambitious','Team spirited','Collaborative','Hard working','Honest','Innovative','Autonomous','Passionate',
           'Outgoing','Proactive','Creative','Motivated','Determined','Versatile','Goal Orientated','Results Focused',
           'Reliable','Problem Solving','Well Organised','Attention to detail', 'Authentic', 'Best of breed','Client focused',
          'Committed', 'Communicative','Core competencies','Customer focused', 'Customer service focused','Dedicated','Dependable',
          'Detail orientated','Direct','Driven','Dynamic','Easy-going', 'Educated', 'Energetic','Enthusiastic','Excellent communicator',
          'Excellent interpersonal skills','Extensive experience','Expert', 'Familiar with', 'Fast learner', 'Fast-paced environment','Flexible','Full of beans',
          'Give it my all', 'Gives 110%', 'Go getter', 'Goal directed','Goal driven', 'Goal setter','Goes above and beyond','Goes the extra mile',
          'Go-to person','Hands-on','Highly experienced','Highly focused', 'Highly organized','Innovative approach','Inspiring',
          'Leader','Loyal','Motivates others','Oversaw a project','People person','Play to strengths','Practical','Proactive',
          'Professional','Professional development','Proven track record','Punctual','Rapport builder','Relationship builder',
          'Results directed','Results driven','Results orientated', 'Sales driven','Seasoned','Self motivated',
          'Self starter','Sincere','Skilled','Specialize','Straightforward','Strategic thinker','Studious','Synergy',
          'Task driven','Task orientated','Team leader','Team player','Teamwork','Think outside the box','Thought leader','Transferable skills',
          'Trustworthy','Upbeat','Works well in a team','Works well under pressure']

# 11. Buzzwords and Clichés
def find_buzzwords(clean_resume):

  resume_buzzwords = [phrase for phrase in buzzwords if phrase.lower() in clean_resume.lower()]
  return resume_buzzwords

# 12. Sections

def find_sections(doc):
    sections = ['Name','Location','Graduation Year','Job Specific Skills', 'Email Address', 'Companies worked at', 'Designation', 'College Name']
    count = 0
    checked_sections = set()
    for ent in doc.ents:
        if ent.label_ in sections and ent.label_ not in checked_sections and ent.text.strip() != '':
            checked_sections.add(ent.label_)
            count += 1
    return count

# 13. Date Consistency

def check_date_consistency(dates):
    formats = [
        r'\d{4}-\d{4}',  # e.g., 2016-2017
        r'\w+ \d{4} - \w+ \d{4}',  # e.g., June 2016 - June 2017
        r'\w{3} \d{4} - \w{3} \d{4}',  # e.g., Jun 2016 - Jun 2017
        r'\d{2}/\d{4} to \d{2}/\d{4}',  # e.g., 06/2016 to 06/2017
        r'\w+ \d{4} - \w+ \d{4}',  # e.g., Summer 2016 - Autumn 2017
    ]

    format_matches = [0] * len(formats)

    for date in dates:
        for i, pattern in enumerate(formats):
            print(date)
            if re.fullmatch(pattern, date):
                format_matches[i] += 1

    return all(count == 0 or count == len(dates) for count in format_matches)


def output_score(resume):

  # overall CV score out of 100
  overall_score = 0

  impact_score = 0
  # read raw resume text
  resume_text = read_resume(resume)[0]

  # clean resume
  clean_resume = clean_CV_simple(resume_text)

  ### Start of IMPACT ### 50 marks total

  # 1. get the quantifiable metrics
  percentages = get_percentages(clean_resume)

  impact_score += min(10,len(percentages))
  print("percentages", len(percentages))

  # 2. Action verb use
  # Helper function to retrieve all the verbs from the resume
  resume_verbs = get_verbs(clean_resume)

  action_verbs = action_verb_use(resume_verbs)

  impact_score += min(len(action_verbs),10)
  print("action verb use", len(action_verbs))

  # 2.1 Strong verb use
  strong_verbs = strong_verbs_resume(resume_verbs)

  impact_score += min(10, len(strong_verbs))
  print("strong verbs", len(strong_verbs))

  # 3. Accomplishments
  accomplishments_resume = accomplishments(clean_resume)

  impact_score += max(0, 10 - len(accomplishments_resume))
  print("accomplishments", 10-len(accomplishments_resume))

  # 4. Repetition

  repetition_resume = repetition(resume_verbs)

  impact_score += max(0, 10 - len(repetition_resume))
  print("repetition", 10-len(repetition_resume))

  #### End Of IMPACT ###

  ### Start of BREVITY ### 40 marks

  brevity_score = 0
  # 5. Resume Length

  resume_pages = read_resume(resume)[2]
  resume_word_count = read_resume(resume)[1]

  if resume_pages < 3:
    brevity_score += 5
  if resume_word_count >= 450  and resume_word_count <= 900:
    brevity_score += 5


  print("resume length", brebity_score)

  # 6. Filler Words

  filler_words_resume = filler_words(clean_resume)

  brevity_score += max(0, 10 - len(filler_words_resume))

  print("filler words", 10-len(filler_words_resume))

  # 7. Total Bullet Points
  bullet_points_resume = total_bullet_points(resume_text)

  if bullet_points_resume >=12 and bullet_points_resume <=20:
    brevity_score += 10
  elif bullet_points_resume < 12:
    brevity_score += max(0, 10 - bullet_points_resume)
  else:
    brevity_score += max(0, abs(20-bullet_points_resume))

  print("total bullet points", brevity_score)

  # 8. Bullet Point Length

  bullet_point_length = count_bullet_word_lengths(resume_text)[0]
  correct_length = 0
  wrong_length = 0

  for length in bullet_point_length:
    if length >= 10 and length <=30:
      correct_length += 1
    else:
      wrong_length += 1


  brevity_score += min(10, correct_length-wrong_length)

  print("bullet points length", correct_length,wrong_length)

  ### Start of STYLE ### 51 marks

  style_score = 0

  # 9. Active voice
  passive_voice = detect_passive_voice(clean_resume)

  style_score += max(0, 10 - passive_voice)

  print("passive voice", 10-passive_voice)

  # 10. Personal Pronouns
  personal_pronouns = find_personal_pronouns(clean_resume)

  style_score += min(0, 5-len(personal_pronouns))
  print("personal pronouns", 5-len(personal_pronouns))

  # 11. Buzzwords and Clichés
  buzzwords = find_buzzwords(clean_resume)

  style_score += min(0, 10-len(buzzwords))

  print("buzzwords", 10-len(buzzwords))

  # 12. Sections

  doc = nlp(clean_resume)   # Load NER model

  resume_sections = find_sections(doc)

  style_score += len(resume_sections) * 2

  print("resume sections", len(resume_sections))

  # 13. Date Consistency

  years_experience = []
  for ent in doc.ents:
    if ent.label_ == "Years of Experience":
      years_experience.append(ent.text)

  consistent = check_date_consistency(years_experience)

  if consistent:
    style_score += 5

  print("date consistency", consistent)

  overall_score = (impact_score + brevity_score + style_score)*100/138
  return impact_score,brevity_score,style_score,overall_score


inputs = [
    File(label="Upload your CV in PDF format")
]

outputs = [
    gr.Text(label = "Impact Score"),
    gr.Text(label="Brevity Score"),
    gr.Text( label="Style Score"),
    gr.Text( label="Overall Score")
]

interface = gr.Interface(
    fn=output_score,
    inputs=inputs,
    outputs=outputs
)

interface.launch(share = True,debug = True)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
<ipython-input-19-4b0a3689bb74>:396: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  File(label="Upload your CV in PDF format")
<ipython-input-19-4b0a3689bb74>:396: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  File(label="Upload your CV in PDF format")
<ipython-input-19-4b0a3689bb74>:396: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  File(label="Upload your CV in PDF f

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

percentages 2


/usr/local/lib/python3.10/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


action verb use 4
strong verbs 10
accomplishments 10
repetition 4
resume length 5
filler words 9
total bullet points 24
bullet points length 6 3
passive voice 8
personal pronouns 4
buzzwords 8
resume sections 8
date consistency True
percentages 2


/usr/local/lib/python3.10/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


action verb use 4
strong verbs 10
accomplishments 10
repetition 4
resume length 5
filler words 9
total bullet points 24
bullet points length 6 3
passive voice 8
personal pronouns 4
buzzwords 8
resume sections 8
date consistency True
percentages 0


/usr/local/lib/python3.10/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


action verb use 4
strong verbs 27
accomplishments 9
repetition -3
resume length 10
filler words 9
total bullet points 24
bullet points length 0 0
passive voice 9
personal pronouns -3
buzzwords 0
resume sections 7
June 2010 – Present
June 2010 – Present
June 2010 – Present
June 2010 – Present
June 2010 – Present
April 2010 – June 2010
April 2010 – June 2010
April 2010 – June 2010
April 2010 – June 2010
April 2010 – June 2010
date consistency True
